<center><img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks&nbsp;">unSkript Runbooks&nbsp;<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective"><strong>Objective</strong><a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<strong>Get publicly accessible AWS RDS DB instances and change them to private.</strong></div>
</center><center>
<h2 id="Publicly-Accessible-Amazon-RDS-Instances">Secure Publicly Accessible Amazon RDS Instances<a class="jp-InternalAnchorLink" href="#Publicly-Accessible-Amazon-RDS-Instances" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<ol>
<li>AWS Get Publicly Accessible RDS Instances</li>
<li>Change public access to private</li>
</ol>

In [ ]:
if rds_instances and not region:
    raise SystemExit("Provide a region for the RDS Instances!")
if region == None:
    region = ""

<h3 id="AWS-Get-Publicly-Accessible-RDS-Instances"><a id="1" target="_self" rel="nofollow"></a>AWS Get Publicly Accessible RDS Instances</h3>
<p>Using unSkript's <strong>AWS Get Publicly Accessible RDS Instances</strong>&nbsp;action, we will get all the publicly accessible instances from RDS instances.</p>
<blockquote>
<p>Input parameters: <code>region</code></p>
</blockquote>
<blockquote>
<p>Output variable: <code>rds_instances</code></p>
</blockquote>

In [ ]:
##
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
import pprint
from typing import Optional, Tuple
from pydantic import BaseModel, Field
from unskript.legos.utils import CheckOutput
from unskript.legos.aws.aws_list_all_regions.aws_list_all_regions import aws_list_all_regions
from unskript.connectors.aws import aws_get_paginator


from beartype import beartype
@beartype
def aws_get_publicly_accessible_db_instances_printer(output):
    if output is None:
        return

    if isinstance(output, CheckOutput):
        print(output.json())
    else:
        pprint.pprint(output)


@beartype
def aws_get_publicly_accessible_db_instances(handle, region: str = "") -> Tuple:
    """aws_get_publicly_accessible_db_instances Gets all publicly accessible DB instances

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region of the RDS.

        :rtype: CheckOutput with status result and list of publicly accessible RDS instances.
    """
    result = []
    all_regions = [region]
    if not region:
        all_regions = aws_list_all_regions(handle)
    for reg in all_regions:
        try:
            ec2Client = handle.client('rds', region_name=reg)
            response = aws_get_paginator(ec2Client, "describe_db_instances", "DBInstances")
            for db in response:
                db_instance_dict = {}
                if db['PubliclyAccessible']:
                    db_instance_dict["region"] = reg
                    db_instance_dict["instance"] = db['DBInstanceIdentifier']
                    result.append(db_instance_dict)
        except Exception:
            pass

    if len(result) != 0:
        return (False, result)
    return (True, None)


task = Task(Workflow())
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "not rds_instances",
    "condition_result": true
    }''')
task.configure(credentialsJson='''{\"credential_type\": \"CONNECTOR_TYPE_AWS\"}''')

task.configure(outputName="public_rds_instances")

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_publicly_accessible_db_instances, lego_printer=aws_get_publicly_accessible_db_instances_printer, hdl=hdl, args=args)

<h3 id="Modify-Output">Create List of Public RDS Instances</h3>
<p>In this action, we modify the output from step 1 and return a list of dictionary items for the publicly accessible RDS instances.</p>
<blockquote>
<p><strong>Output variable:</strong> all_public_instances</p>
</blockquote>

In [ ]:
all_public_instances = []
try:
    if public_rds_instances[0] == False:
        for instance in public_rds_instances[1]:
            all_public_instances.append(instance)
except Exception as e:
    if rds_instances:
        for ins in rds_instances:
            data_dict = {}
            data_dict["region"] = region
            data_dict["instance"] = ins
            all_public_instances.append(data_dict)
    else:
        raise Exception(e)
print(all_public_instances)

<h3 id="Change-the-public-access-to-private">Change the public access to private</h3>
<p>Using unSkript's Modify Publicly Accessible RDS Instances action we will modify the access to all the publicly accessible instances from the <em>public</em> to <em>private</em>.</p>
<blockquote>
<p>This action takes the following parameters: <code>region</code>, <code>db_instance_identifier</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2023 unSkript, Inc
# All rights reserved.
##
from pydantic import BaseModel, Field


from beartype import beartype
@beartype
def aws_make_rds_instance_not_publicly_accessible_printer(output):
    if output is None:
        return
    print(output)


@beartype
def aws_make_rds_instance_not_publicly_accessible(handle, db_instance_identifier: str, region: str) -> str:
    """
    aws_make_rds_instance_not_publicly_accessible makes the specified RDS instance not publicly accessible.

    :type handle: object
    :param handle: Object returned from task.validate(...).

    :type db_instance_identifier: string
    :param db_instance_identifier: Identifier of the RDS instance.

    :type region: string
    :param region: Region of the RDS instance.

    :rtype: Response of the operation.
    """
    try:
        rdsClient = handle.client('rds', region_name=region)
        rdsClient.modify_db_instance(
            DBInstanceIdentifier=db_instance_identifier,
            PubliclyAccessible=False
        )
    except Exception as e:
        raise e
    return f"Public accessiblilty is being changed to False..."




task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(credentialsJson='''{\"credential_type\": \"CONNECTOR_TYPE_AWS\"}''')
task.configure(inputParamsJson='''{
    "db_instance_identifier": "iter.get(\\"instance\\")",
    "region": "iter.get(\\"region\\")"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "all_public_instances",
    "iter_parameter": ["db_instance_identifier","region"]
    }''')
task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "len(all_public_instances)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_make_rds_instance_not_publicly_accessible, lego_printer=aws_make_rds_instance_not_publicly_accessible_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion<a class="jp-InternalAnchorLink" href="#Conclusion" target="_self">&para;</a></h3>
<p>In this Runbook, we demonstrated the use of unSkript's AWS actions. This runbook help to find publicly accessible RDS instances and change it to private. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">https://us.app.unskript.io</a></p>